In [1]:
import hopsworks
from sklearn.neighbors import KNeighborsClassifier
import hsfs
import numpy as np
import datetime
import os

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: quY2gxHAowy7kpmZ.iV0LYHaixcMXqUpIlMXA3qOmiIyM3Y63mAAk4QgKzxi4eZ881wMTiB5CGCjOLcZZ
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/135
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
# Uncomment this cell and fill in details if you are running your own Hopsworks cluster
# key=""
# with open("api-key.txt", "r") as f:
#     key = f.read().rstrip()
# os.environ['HOPSWORKS_PROJECT']="cjsurf"
# os.environ['HOPSWORKS_HOST']="35.187.178.84"
# os.environ['HOPSWORKS_API_KEY']=key

In [ ]:
project = hopsworks.login()
fs = project.get_feature_store()

In [2]:
mr = project.get_model_registry()
model = mr.get_model("cjsurf_model", version=1)
model_dir = model.download()

Connected. Call `.close()` to terminate connection gracefully.

In [3]:
import joblib
model = joblib.load(model_dir + "/knn_model.pkl")

In [ ]:
fv = fs.get_feature_view("lahinch_surf")
fv.init_serving()

In [ ]:
key = {"beach_id" : 1}
vector = fv.get_feature_vector(key)
print(vector)

In [ ]:
feature_vectors = []
feature_vectors.append(vector[1:4])
for i in range(5,len(vector),4):
    feature_vectors.append(vector[i:i+3])
    
dt = vector[4]

In [ ]:
row_list=[]
columns=["beach_id","hits_at", "wave_height"]
for i in range(0,len(feature_vectors)):
    arr=[]
    arr.append(feature_vectors[i])
    res = model.predict(arr)
    row = []
    row.append(1) #beach_id
    hour = datetime.timedelta(0, (1*3600*i))
    ts = dt + hour
    dt_str = ts.strftime("%m/%d/%Y, %H:%M") #:%S
    row.append(dt_str)
    row.append(res[0])
    row_list.append(row)

row_list

In [ ]:
import pandas as pd

df = pd.DataFrame(row_list, columns=columns)

df

In [ ]:
fs = project.get_feature_store()

preds_fg = fs.get_or_create_feature_group(name="wave_predictions",
                version=1,
                primary_key=["beach_id", "hits_at"],
                description="Lahinch surf height predictions",
                online_enabled=True
                )
preds_fg.insert(df)   